In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import gc
import os 

# clear cuda memory and collect garbage
gc.collect()
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: ',device)

x=np.load('scaled_spec_resampled_array.npy')
x=x[:,724:1324,:]
y=np.load('labels_array.npy')-1 
x = x.reshape(x.shape[0], 1, x.shape[1], x.shape[2])

print(x.shape, y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Device:  cuda:0
(1754, 1, 600, 80) (1754,)


In [6]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    return val_loss, val_acc

def train(model, train_loader, test_loader, criterion, optimizer, epochs, checkpointname):
    model.train()
    running_loss = 0.0
    epoch_bar = tqdm(range(epochs), position=0)

    best_val_acc = 0
    val_loss, val_acc = test(model, test_loader, criterion)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), checkpointname)

    for epoch in epoch_bar:

        batch_bar=tqdm(enumerate(train_loader, 0), total=len(train_loader), position=1, leave=False)

        for i, data in batch_bar:
        #for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            batch_bar.set_description('Train loss: %.3f' % (loss.item()))

        train_loss = running_loss / len(train_loader)
        epoch_bar.set_description('Train loss: %.3f' % train_loss)
        val_loss, val_acc = test(model, test_loader, criterion)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), checkpointname)

        print('Epoch: %d, Train Loss: %.3f, Val Loss: %.3f, Val Acc: %.3f' % (epoch, train_loss, val_loss, val_acc))
    return model


In [7]:
resnet18 = models.resnet18(pretrained=True)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet18.fc = nn.Linear(in_features=512, out_features=6, bias=True)    

resnet18 = resnet18.to(device)

#check for checkpoint save and load resnet18_smalldata.pth
if os.path.exists('resnet18_smalldata.pth'):
    resnet18.load_state_dict(torch.load('resnet18_smalldata.pth'))
    print('Model loaded')
else:
    print('No model found, loading pretrained model')

#freeze every layer except the first conv1 layer and the last fc layer, to adapt the model to our data
for name, param in resnet18.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True

optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

resnet18=train(resnet18, train_loader,test_loader, criterion, optimizer, 100, 'resnet18_smalldata.pth')

c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded


Train loss: 0.391:   1%|          | 1/100 [00:03<05:17,  3.21s/it]

Epoch: 0, Train Loss: 0.391, Val Loss: 0.486, Val Acc: 0.818


Train loss: 0.662:   2%|▏         | 2/100 [00:05<04:38,  2.84s/it]

...

Epoch: 97, Train Loss: 21.375, Val Loss: 0.495, Val Acc: 0.852


Train loss: 21.568:  99%|█████████▉| 99/100 [04:45<00:03,  3.11s/it]

Epoch: 98, Train Loss: 21.568, Val Loss: 0.558, Val Acc: 0.812


Train loss: 21.788: 100%|██████████| 100/100 [04:48<00:00,  2.89s/it]

Epoch: 99, Train Loss: 21.788, Val Loss: 0.529, Val Acc: 0.838


In [8]:
resnet34 = models.resnet34(pretrained=True)
resnet34.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet34.fc = nn.Linear(in_features=512, out_features=6, bias=True)    

#check for checkpoint save and load resnet34.pth
if os.path.exists('resnet34_smalladata.pth'):
    resnet34.load_state_dict(torch.load('resnet34_smalladata.pth'))
    print('Model loaded')
else:
    print('No model found, loading pretrained model')

resnet34 = resnet34.to(device)

#freeze every layer except the first conv1 and last fc layers of resnet50
for name, param in resnet34.named_parameters():
    if name not in ['conv1.weight', 'fc.weight', 'fc.bias']:
        param.requires_grad = False
    else:
        param.requires_grad = True

optimizer = optim.Adam(resnet34.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

resnet34=train(resnet34, train_loader,test_loader, criterion, optimizer, 100, 'resnet34_smalladata.pth')

c:\Users\giaco\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded


Train loss: 0.219:   1%|          | 1/100 [00:05<08:29,  5.14s/it]

Epoch: 0, Train Loss: 0.219, Val Loss: 0.548, Val Acc: 0.803


Train loss: 0.402:   2%|▏         | 2/100 [00:09<07:47,  4.77s/it]

...

Epoch: 97, Train Loss: 8.805, Val Loss: 0.580, Val Acc: 0.838


Train loss: 8.858:  99%|█████████▉| 99/100 [08:03<00:04,  4.72s/it]

Epoch: 98, Train Loss: 8.858, Val Loss: 0.529, Val Acc: 0.829


Train loss: 8.903: 100%|██████████| 100/100 [08:07<00:00,  4.88s/it]

Epoch: 99, Train Loss: 8.903, Val Loss: 0.564, Val Acc: 0.832
